Downloading Libraries

In [ ]:
!pip install transformers datasets nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

Context Mapping

In [16]:
import json
import pandas as pd

# Load JSON file
with open("/content/sarcasm_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Convert JSON to a structured list
rows = []
for key, value in data.items():
    context_mapping = [
        f"{speaker}: {sentence}" for speaker, sentence in zip(value["context_speakers"], value["context"])
    ]

    rows.append({
        "ID": key,
        "Utterance": value["utterance"],
        "Speaker": value["speaker"],
        "Context": " || ".join(value["context"]),
        "Context_Speakers": " || ".join(value["context_speakers"]),
        "Context_Mapped": " || ".join(context_mapping),
        "Show": value["show"],
        "Sarcasm": int(value["sarcasm"])
    })

# Convert to DataFrame
df = pd.DataFrame(rows)

print("✅ JSON to CSV conversion complete!")


✅ JSON to CSV conversion complete!


Context Summarization

In [18]:
from transformers import pipeline
from datasets import Dataset

# Initialize Summarizer (GPU Optimized)
summarizer = pipeline("summarization", model="knkarthick/MEETING_SUMMARY", device=0)
dataset = Dataset.from_pandas(df)

# Summarization Function
def batch_summarize(batch):
    texts = batch["Context_Mapped"]
    summaries = []

    for text in texts:
        max_len = max(10, int(len(text.split()) * 0.7))  # Ensure max_length < input_length
        summary = summarizer(text, max_length=max_len, min_length=10, do_sample=False)
        summaries.append(summary[0]["summary_text"])

    return {"Context_Summary": summaries}



# Apply Summarization
dataset = dataset.map(batch_summarize, batched=True, batch_size=16)
df = pd.DataFrame(dataset)

print("✅ Summarization complete!")
df.to_csv("sarcasm_data.csv", index=False)


Device set to use cuda:0


Map:   0%|          | 0/690 [00:00<?, ? examples/s]

Your max_length is set to 10, but your input_length is only 7. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)
Your max_length is set to 10, but your input_length is only 7. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)
Your max_length is set to 10, but your input_length is only 8. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)
Your max_length is set to 10, but your input_length is only 8. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)
Your max_length 

✅ Summarization complete!
